In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import math

In [ ]:
def ellipse_arc(
    x_center = 0,
    y_center = 0,
    a = 1,
    b = 1,
    start_angle = 0,
    end_angle = 2 * np.pi,
    N = 100,
    closed = False
):
    t = np.linspace(start_angle, end_angle, N)
    x = x_center + a * np.cos(t)
    y = y_center + b * np.sin(t)
    path = f'M {x[0]}, {y[0]}'
    for k in range(1, len(t)):
        path += f'L{x[k]}, {y[k]}'
    if closed:
        path += ' Z'
    return path

In [ ]:
df = pd.read_csv("database.csv")
df["y"] = 1 - df["y"]

In [ ]:
df_grouped = df.groupby("id")
df["oposite"] = df["y"] - df_grouped["y"].shift(1)
df["adjacent"] = df["x"] - df_grouped["x"].shift(1)
df["distance"] = (df["adjacent"] ** 2 + df["oposite"] ** 2) ** (1. / 2.)
moves = df[df["distance"] > 0].reset_index()

In [6]:
moves["sin"] = moves["oposite"] / moves["distance"]
moves["cos"] = moves["adjacent"] / moves["distance"]
moves["tan"] = moves["oposite"] / moves["adjacent"]
moves["angle"] = np.arccos(moves["cos"])
moves["fromward"] = moves.groupby("id")["angle"].shift(1)
moves["toward"] = moves.groupby("id")["angle"].shift(-1)

In [7]:
moves["path"] = np.vectorize(ellipse_arc)(
    x_center = moves["x"] + moves["adjacent"] / 2,
    y_center = moves["y"] + moves["oposite"] / 2,
    a = moves["adjacent"] / 2,
    b = moves["oposite"] / 2,
    start_angle = moves["fromward"],
    end_angle = moves["toward"],
    N = np.round(moves["distance"] * 1000).astype(int)
)

In [9]:
# DESPLAÇAMENTS

fig = go.Figure()
fig.add_trace(go.Scatter(
        name = "desplaçaments",
        mode = "lines",
        x = df["x"],
        y = df["y"],
        line = dict(
            color = "#333333"
        )
    )
)

fig.update_yaxes(
    visible = False
)

fig.update_xaxes(
    visible = False
)

fig.update_layout(
    height = 600,
    width = 800,
    title_text = "Desplaçaments",
    paper_bgcolor = "rgba(256, 256, 256, 256)",
    plot_bgcolor = "rgba(230, 230, 230, 256)"
)

In [12]:
# DESPLAÇAMENTS PER PESONATGES

fig = make_subplots(rows = 5, cols = 2)
for n, player in enumerate(pd.unique(df["player"])):
    fig.add_trace(
        go.Scatter(
            name = player,
            x = df[df["player"] == player]["x"],
            y = df[df["player"] == player]["y"]
        ),
        row = n // 2 + 1,
        col = n % 2 + 1
    )

fig.update_yaxes(
    visible = False
)

fig.update_xaxes(
    visible = False
)

fig.update_layout(
    height = 1500,
    title_text = "Desplaçaments dels personatges",
    paper_bgcolor = "rgba(256, 256, 256, 256)",
    plot_bgcolor = "rgba(230, 230, 230, 256)"
)

fig.show()

In [13]:
# OCUPACIÓ DE L'ESPAI

data = df[["x", "y"]].reset_index()
data["x"] = (data["x"] // .01).astype(int)
data["y"] = (data["y"] // .01).astype(int)

matrix = np.matrix([[data[(data["x"] == x) & (data["y"] == y)].shape[0] for x in range(100)] for y in range(100)])

fig = go.Figure(
    data = go.Heatmap(
        z = np.power(matrix, 1 / 3),
        x = list(range(100)),
        y = list(range(100))
    )
)

fig.update_yaxes(
    visible = False
)

fig.update_xaxes(
    visible = False
)

fig.update_layout(
    height = 600,
    width = 800,
    title_text = "Presència",
    paper_bgcolor = "rgba(256, 256, 256, 256)",
    plot_bgcolor = "rgba(230, 230, 230, 256)"
)

fig.show()

In [14]:
# RITMES
ryt = df[["time", "distance"]].copy()
ryt["time"] = (ryt["time"] - min(ryt["time"])).astype(int)
ryt = ryt.sort_values(by = "time").reset_index()
ryt = ryt.groupby("time").sum().reset_index()[["time", "distance"]]

fig = go.Figure(
    data = go.Scatter(
        mode = "lines",
        x = ryt["time"],
        y = ryt["distance"],
        fill = "tozeroy",
        fillcolor = "#333333",
        line = dict(
            width = .5,
            color = "#333333"
        )
    )
)

fig.update_yaxes(
    visible = False
)

fig.update_layout(
    height = 400,
    title_text = "Ritme"
)

fig.show()

In [16]:
fig = make_subplots(
    rows = 5,
    cols = 2,
    shared_yaxes = True
)

for n, player in enumerate(pd.unique(df["player"])):
    ryt = df[df["player"] == player][["time", "distance"]]
    ryt["time"] = (ryt["time"] - min(df["time"])).astype(int)
    ryt = ryt.sort_values(by = "time").reset_index()
    ryt = ryt.groupby("time").sum().reset_index()[["time", "distance"]]

    fig.add_trace(go.Scatter(
            name = player,
            mode = "lines",
            x = ryt["time"],
            y = ryt["distance"],
            fill = "tozeroy",
            line = dict(
                width = .5
            )
        ),
        row = n // 2 + 1,
        col = n % 2 + 1
    )
    
fig.update_yaxes(
    visible = False
)

fig.update_xaxes(
    visible = True
)

fig.update_layout(
    height = 1500,
    title_text = "Ritmes",
    paper_bgcolor = "rgba(256, 256, 256, 256)",
    plot_bgcolor = "rgba(230, 230, 230, 256)"
)
    
fig.show()

In [ ]:
px.scatter(
    df,
    x = "x",
    y = "y",
    animation_frame = "time",
    animation_group = "player",
    color = "player"
)